# Figures

## Inputs

- X, y
- test/train splits
- Model scores
- Feature importance scores for each model
- SHAP scores for each model

## Outputs

- Figure 2: model performance + brain networks
- Figure 3: feature importance
- Figure 4: robustness of feature importance
- Figure 5: misclassified subjects

## Requirements

```bash
mamba create -f enviroment.yml

mamba install seaborn matplotlib shap

```

## Setup

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

sns.set('paper', style='whitegrid')

In [ ]:
n_subjects = ...
X = ...
y = ...
grid_cv_results_ = ...
cv_test_size = ...
scoring = ...
grid_cv = ...
perm_scores = ...
perm_obs_scores = ...
perm_pvalue = ...
perm_imp_scores = ...
shap_values = ...
shap_importances = ...
shap_y_pred = ...
shap_y_test = ...
feature_names = ...

In [ ]:
# PLOT MODEL PERFORMANCE

def _create_label(params):
    return f'$\\bf{{{params[0]}}}$\n{params[1]}'.replace('_2mm', '').replace('_','\\_')

sns.set('paper', 'ticks')

# prep data
scores = grid_cv_results_.copy()
scores['label'] = scores['params'].apply(lambda p: _create_label(list(p.values())))
split_score_cols = [c for c in scores.columns if 'split' in c]
scores['cv_test_score'] = scores[split_score_cols].apply(lambda x: list(x), axis=1)
scores = scores.explode('cv_test_score').reset_index(drop=True)
scores = scores.sort_values('rank_test_score', ascending=False)

_, ax = plt.subplots(figsize=(len(grid_cv_results_['params']) * 1.1, 5))

# score curve + CI
sns.lineplot(data=scores, x='label', y='cv_test_score', lw=3, sort=False, ax=ax)

# blue dots
sns.scatterplot(data=scores, x='label', y='mean_test_score', marker='o', s=100, ax=ax, label='_nolegend_')

if plot_chance:
    # simulate chance
    chance_scores = {}
    for model_name in scores['label'].unique():
        chance_model = DummyClassifier(strategy='stratified').fit(X, y)
        _chance_scores = cross_val_score(chance_model, X, y, cv=grid_cv, scoring=scoring)
        chance_scores[model_name] = _chance_scores

    chance_scores = pd.DataFrame(chance_scores)
    chance_scores = chance_scores.melt(value_name=scoring, var_name='label')

    # plot chance
    sns.lineplot(data=chance_scores, x='label', y=scoring, lw=2, sort=False, ax=ax, color='red')

# legend
ax.legend(['average', '95% CI', 'chance'],
            title_fontproperties={'weight': 'bold', 'size': 'x-large'},
            prop={'size': 'x-large'}, loc='upper left',
            title=f'{cv.n_splits} $\\times$ {n_subjects // cv.test_size}-fold CV')

# tweaks
ax.set_xlabel('')
ax.set_ylabel(scoring, fontsize='xx-large')
plt.xticks(rotation=45, ha='right', fontsize='x-large', rotation_mode='anchor')
plt.suptitle(f'classification {scoring} (validation set)', fontsize='xx-large', y=.95)
plt.grid(axis='y')

plt.show()

In [ ]:
# SIGNIFICANCE TESTING

scores = grid_cv_results_.copy()
scores['label'] = scores['params'].apply(lambda p: ' '.join(p.values()))
split_score_cols = [c for c in scores.columns if 'split' in c]
scores['cv_test_score'] = scores[split_score_cols].apply(lambda x: list(x), axis=1)
scores = scores.explode('cv_test_score').reset_index(drop=True)
scores = scores.sort_values('rank_test_score', ascending=False)

chance_model = DummyClassifier(strategy='stratified')
chance_scores = cross_val_score(chance_model, X, y, cv=grid.cv, scoring=grid.scoring)

# visible_labels = ['chance','dosenbach2010 tangent', 'dosenbach2010 precision', 'dosenbach2010'
#        'dosenbach2010 partial correlation']

# scores = scores.query('label in @visible_labels')


scores = pd.concat([pd.DataFrame({
    'cv_test_score': chance_scores,
    'label': '$\\bf{chance}$'
}), scores]).reset_index()

_, ax = plt.subplots(figsize=(8, 5))


sns.set('paper', style='white')
g = sns.barplot(data=scores[::-1], y='label', x='cv_test_score', orient='h', ci=95, capsize=.3, ax=ax, palette='RdBu_r')
ax.set_xlabel(scoring, fontsize='x-large')
ax.set_yticklabels(ax.get_yticklabels(), fontsize='x-large')

ax.set_ylabel('atlas and connectivity', fontsize='x-large', labelpad=20)


from statannotations.Annotator import Annotator

annotator = Annotator(g, [("dosenbach2010 partial correlation", '$\\bf{chance}$')], data=scores, y='label', x='cv_test_score', orient='h')
annotator.configure(test='t-test_ind', text_format='star', loc='inside',)
annotator.apply_and_annotate()


plt.show()


In [ ]:
# PLOTS
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

g = sns.FacetGrid(scores, row="label", hue="label", aspect=12, height=.5)

# Draw the densities in a few steps
g.map(sns.kdeplot, "cv_test_score",fill=True, alpha=1, linewidth=1)
g.map(sns.kdeplot, "cv_test_score", color="w", lw=1.5)

# passing color=None to refline() uses the hue mapping
g.refline(y=0, linewidth=1, linestyle="-", color=None, clip_on=False)

import matplotlib.patheffects as PathEffects

def label(x, color, label):
    ax = plt.gca()
    txt = ax.text(0, .2, label.replace('_2mm',''), color=color, weight='bold', fontsize=10,
            ha="left", va="center", transform=ax.transAxes)
    txt.set_path_effects([PathEffects.withStroke(linewidth=3, foreground='w')])


g.map(label, "label")

# Set the subplots to overlap
g.figure.subplots_adjust(hspace=-.2)

# Remove axes details that don't play well with overlap
g.set_titles("")
g.set(yticks=[], ylabel='', xlabel=scoring, xlim=(0, 1))
g.despine(bottom=True, left=True)

# g.figure.gca().axvline(x=0.8, color='brown', linestyle='-', linewidth=2)

plt.show()

# BOX PLOT
sns.boxplot(data=scores, x='cv_test_score', y='label', whis=0, fliersize=0, saturation=1)
sns.stripplot(x="cv_test_score", y="label", data=scores, size=4, color=".3", linewidth=0)

plt.show()

In [ ]:

# Now plot permuted scores along with the cross-validated scores

sns.set('paper','white')
_, ax = plt.subplots(figsize=(8,5))

# permutation scores
sns.histplot(perm_scores, ax=ax, color=(.86, .26, .46), lw=0, binwidth=.15, stat='density', label='permuted labels',alpha=.7)
sns.kdeplot(perm_scores, ax=ax, color=(.86, .26, .46), lw=2, cut=0, bw_method=.5)
ax.axvline(perm_scores.mean(), color=(.86, .26, .46), ls='--', lw=2)

# observation scores
sns.histplot(perm_obs_scores, ax=ax, color=(.11, .53, .89), lw=0, binwidth=.15, stat='density',  label='observed labels', alpha=.8)
g = sns.kdeplot(perm_obs_scores, ax=ax, color=(.11, .53, .89), lw=2, fill=True, cut=0, bw_method=.5)
ax.axvline(perm_obs_scores.mean(), color=(.11, .53, .89), ls='--', lw=2)

# tweaks
plt.xlabel('accuracy', fontsize='xx-large')
plt.ylabel('density', fontsize='xx-large')
plt.xticks(fontsize='x-large')

plt.suptitle(f'Permutation Testing (N={len(perm_scores)})', x = .31, y=.93, fontsize='xx-large')
plt.legend(loc='upper right', fontsize='x-large')

plt.text(perm_obs_scores.mean() + .01,  2.5, f'p–value={perm_pvalue:.3f}', fontsize='x-large')

plt.show()

In [ ]:
_, ax = plt.subplots(1,1,figsize=(5,7))
sns.barplot(data=perm_imp_scores,
            orient='horizontal',
            capsize=.3,
            errwidth=2,
            color=(.11, .53, .89),
            ax=ax)

plt.xticks(fontsize='x-large')
plt.yticks(fontsize='x-large')
plt.xlabel('Permutation importance (impact on model accuracy)', fontsize='xx-large')
plt.show()

In [ ]:
shap.summary_plot(shap_values, max_display=21, show=True, plot_type='violin')


In [ ]:

_, ax = plt.subplots(1,1,figsize=(5,7))
sns.barplot(data=shap_importances[::-1],
            x='shap',
            y='feature_name',
            orient='horizontal',
            capsize=.3,
            errwidth=2,
            color=(.11, .53, .89),
            ax=ax)

plt.xticks(fontsize='x-large')
plt.yticks(fontsize='x-large')
plt.ylabel('')
plt.xlabel('SHAP value (impact on model output)', fontsize='xx-large')
plt.show()

In [ ]:
# DIAGNOSIS OF MISCLASSIFICATION

incorrect_pred_mask = np.where(shap_y_pred != shap_y_test)[0]     # miss classified
correct_pred_mask = np.where(shap_y_pred == shap_y_test)[0]     # correctly classified

sns.set('paper', 'whitegrid')

# DBEUG
# shap.summary_plot(shap_values.values[misclassified_mask],
#                   plot_type='bar', feature_names=feature_names)

missclassified_shaps = np.abs(shap_values.values[incorrect_pred_mask]).mean(axis=0)

plot_data = pd.DataFrame({
    'incorrect_shap': np.abs(shap_values.values[incorrect_pred_mask]).mean(axis=0),
    'correct_shap': np.abs(shap_values.values[correct_pred_mask]).mean(axis=0),
    'feature_name': feature_names
})

plot_data = plot_data.sort_values('correct_shap', ascending=False)
plot_data['incorrect_shap'] = -plot_data['incorrect_shap']

_, ax = plt.subplots(1,1,figsize=(5,7))

sns.barplot(data=plot_data, x='correct_shap', y='feature_name', color='blue', ax=ax, label='correctly classified')
sns.barplot(data=plot_data, x='incorrect_shap', y='feature_name', color='red', ax=ax, label='misclassified')

# ax.text(.05, 7.5, 'Correctly\nClassified', color='blue', fontsize='x-large', ha='left')
# ax.text(-.05, 7, 'Misclassified', color='red', fontsize='x-large', ha='right')

plt.yticks(fontsize='xx-large')
plt.legend(fontsize='x-large', loc='lower right')

for tick in ax.get_yticklabels():
    if '\N{left right arrow}' not in str(tick):
        tick.set_color('lightgray')

ax.set_xticklabels([np.round(np.abs(t), 2) for t in ax.get_xticks()])

ax.set_xlabel('SHAP value (impact on the model output)', fontsize='xx-large')
ax.set_ylabel('feature', fontsize='xx-large')

# plt.suptitle(f'SHAP values for {n_samples} correctly classified subjects sampled from all the cross-validation validation splits.\n', ha='left', x=-.3, y=.9, fontsize=14)
plt.show()